In [1]:
import os
import unicodedata

import fiona
from fiona.crs import to_string

from madmex.models import Tag


# 9) Asignar etiquetas numéricas al shapefile dado que trae etiquetas

In [2]:
path_shapefile_with_classes = '/LUSTRE/MADMEX/training/cultivos/igamma_veracruz/entrega_1a_semana_febrero_2020/'

In [3]:
input_file = path_shapefile_with_classes + 'cultivos2019.shp'

In [4]:
input_file

'/LUSTRE/MADMEX/training/cultivos/igamma_veracruz/entrega_1a_semana_febrero_2020/cultivos2019.shp'

In [5]:
with fiona.open(input_file) as src:
    fc=list(src)
    crs = to_string(src.crs)

In [6]:
fc[0]

{'type': 'Feature',
 'id': '0',
 'properties': OrderedDict([('PuntoID', 1),
              ('Fecha', 'mar-19'),
              ('Municipio', 'Catemaco'),
              ('DistritoDR', 'San Andres Tuxtla'),
              ('Ecoregion', 'Sierra de los Tuxtlas'),
              ('ZonadeVida',
               'Bosque Lluvioso premontano [Cálido - Muy Lluvioso]'),
              ('Cultivo', 'Café'),
              ('Asociació', 'NA'),
              ('X', -95.105416),
              ('Y', 18.341956),
              ('Observacio', 'CERCA PLÁTANO')]),
 'geometry': {'type': 'Point', 'coordinates': (-95.105416, 18.341956)}}

In [7]:
name_attribute='Cultivo'

In [8]:
fc_subset = [{'id': feature['id'],
              'name': feature['properties'][name_attribute],
              'geometry': feature['geometry']} for feature in fc]

In [9]:
fc_subset[0]

{'id': '0',
 'name': 'Café',
 'geometry': {'type': 'Point', 'coordinates': (-95.105416, 18.341956)}}

In [10]:
def normalize_name_classes(string):
    return unicodedata.normalize('NFKD', string).encode('ASCII','ignore').decode('utf-8').lower()

In [11]:
for k in range(len(fc_subset)):
    fc_subset[k]['name'] =  normalize_name_classes(fc_subset[k]['name'])

In [12]:
fc_subset[0]

{'id': '0',
 'name': 'cafe',
 'geometry': {'type': 'Point', 'coordinates': (-95.105416, 18.341956)}}

In [13]:
classes = list(set([feature['name'] for feature in fc_subset]))

In [14]:
classes

['mango',
 'papaya',
 'hule hevea',
 'sorgo',
 'maiz',
 'cafe sol',
 'chayote',
 'pastizal',
 'citrico',
 'cafe',
 'soya',
 'pina',
 'cana',
 'platano',
 'palma de aceite']

In [15]:
len(classes)

15

In [16]:
def add_numeric_code_to_classes(classes):
    return {classes[k]: k for k in range(len(classes))}

In [17]:
classes = add_numeric_code_to_classes(classes)

In [18]:
classes

{'mango': 0,
 'papaya': 1,
 'hule hevea': 2,
 'sorgo': 3,
 'maiz': 4,
 'cafe sol': 5,
 'chayote': 6,
 'pastizal': 7,
 'citrico': 8,
 'cafe': 9,
 'soya': 10,
 'pina': 11,
 'cana': 12,
 'platano': 13,
 'palma de aceite': 14}

# 10) Insertar en madmex_Tag y añadir numeric_code a fc_subset

**Verificar credenciales de la DB a la que se insertará**

In [19]:
scheme_tag = 'agricultura_cultivos2019'

In [20]:
[Tag.objects.get_or_create(scheme=scheme_tag, value=name_class, numeric_code=classes[name_class],color='') for name_class in classes]

[(<Tag: Tag object (12)>, True),
 (<Tag: Tag object (13)>, True),
 (<Tag: Tag object (14)>, True),
 (<Tag: Tag object (15)>, True),
 (<Tag: Tag object (16)>, True),
 (<Tag: Tag object (17)>, True),
 (<Tag: Tag object (18)>, True),
 (<Tag: Tag object (19)>, True),
 (<Tag: Tag object (20)>, True),
 (<Tag: Tag object (21)>, True),
 (<Tag: Tag object (22)>, True),
 (<Tag: Tag object (23)>, True),
 (<Tag: Tag object (24)>, True),
 (<Tag: Tag object (25)>, True),
 (<Tag: Tag object (26)>, True)]

In [21]:
def get_numeric_code(string):
    return classes[string]

In [22]:
fc_subset = [{'id': feature['id'],
               'name': feature['name'],
               'code': get_numeric_code(feature['name']),
               'geometry': feature['geometry']} for feature in fc_subset]

In [23]:
fc_subset[10]

{'id': '10',
 'name': 'pastizal',
 'code': 7,
 'geometry': {'type': 'Point', 'coordinates': (-94.074697074, 17.77950754)}}

In [24]:
fc_subset_schema = {'geometry': 'Point',
                    'properties': {'name': 'str',
                                   'code': 'int'}
                   }

In [26]:
output_file_with_numeric_codes = path_shapefile_with_classes + 'cultivos2019' + '_with_numeric_code.shp'

In [27]:
output_file_with_numeric_codes

'/LUSTRE/MADMEX/training/cultivos/igamma_veracruz/entrega_1a_semana_febrero_2020/cultivos2019_with_numeric_code.shp'

In [28]:
driver='ESRI Shapefile'

In [29]:
layer=os.path.basename(output_file_with_numeric_codes).split('.shp')[0]

In [37]:
layer

'cultivos2019_with_numeric_code.shp'

In [38]:
with fiona.open(output_file_with_numeric_codes, 'w',
                encoding='utf-8',
                driver=driver,
                layer=layer,
                crs=crs,
                schema=fc_subset_schema) as dst:
    [dst.write({'geometry': feature['geometry'],
                'properties': {'name': feature['name'],
                               'code': feature['code']}
                }) for feature in fc_subset]

# 11) Registrar cultivos con etiquetas numéricas en trainclassification con esquema "agricultura_cultivos2019"

In [40]:
%%bash
/home/madmex_user/.local/bin/antares ingest_training_from_vector /LUSTRE/MADMEX/training/cultivos/igamma_veracruz/entrega_1a_semana_febrero_2020/cultivos2019_with_numeric_code.shp --scheme agricultura_cultivos2019 --year 2020 --name cultivos2019_inecol --field code

Command line arguments: {'verbosity': 1, 'settings': None, 'pythonpath': None, 'traceback': False, 'no_color': False, 'force_color': False, 'input_file': '/LUSTRE/MADMEX/training/cultivos/igamma_veracruz/entrega_1a_semana_febrero_2020/cultivos2019_with_numeric_code.shp', 'scheme': 'agricultura_cultivos2019', 'field': 'code', 'name': 'cultivos2019_inecol', 'year': '2020', 'app': False, 'train_interpreted': False, 'scheme_interpreted': None, 'field_interpreted': None, 'dc_tile': None}
/home/madmex_user/.local/lib/python3.6/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
2020-02-10 22:37:28,442 - madmex.wrappers - loggerwriter write: /home/madmex_user/.local/lib/python3.6/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
2020-02-